pip install of torch geometric module for graph neural network usage.

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 427.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.5 MB/s eta 0:00:00


Full imports.

In [ ]:
import os;
import math;
from copy import deepcopy;
import random;
import networkx as nx;
import pandas as pd;
import torch;
from torch.nn import Linear;
from torch.nn.functional import relu;
from torch_geometric.nn import MessagePassing, GATConv;
from torch_geometric.data import Data;
from torch_geometric.loader import DataLoader;
from torch.optim.lr_scheduler import ReduceLROnPlateau;

In [ ]:
from google.colab import drive;
drive.mount('/content/drive');

Mounted at /content/drive


In [ ]:
def load_amino_acid_labels(csv_file, amino_acids):
    """
    Loads amino acid labels from a CSV file.

    Parameters
    ----------
    csv_file : The csv file in the directory to be processed.
    amino_acids : The constant

    Returns
    -------
    Returns the one-hot encoded amino acid features (sequence).

    """

    seq = pd.read_csv(csv_file, header=None);
    # One-hot encoding for amino acid features.
    seq = seq[0].to_numpy();
    aa_to_OH = [];
    for i, j in enumerate(seq):
        aa_to_OH.append(amino_acids[j]);
    aa_tensor = torch.tensor(aa_to_OH, dtype=torch.int);
    return aa_tensor;

In [ ]:
def load_graph(graph_file):
    """
    Loads the molecular graph of the protein from the graph file.

    Parameters
    ----------
    graph_file: The graph file in the directory to be processed.

    Returns
    -------
    Returns the networkx graph containing the nodes and their related edges.

    """
    # Utilises the networkx read_graphml function to extract nodes & edges.
    graph = nx.read_graphml(graph_file);
    node_data = [];
    # Necessary to remove strings from the tensor data.
    for node, coord in graph.nodes(data=True):
        node_index = int(node);
        coords = [coord['coordX'], coord['coordY'], coord['coordZ']];
        # Combines the integer node with its corresponding coordinate floats.
        node_data.append([node_index] + coords);
    # Resulting 32 bit float tensor:
    node_tensor = torch.tensor(node_data, dtype=torch.float);
    edge_data = [];
    edge_weight = [];
    # Similar operation for the edge list.
    for source, target, weight in graph.edges(data=True):
        # Combines the edge with its corresponding weighting.
        edge_data.append([int(source) - 1, int(target) - 1]);
        edge_weight.append(float(weight['weight']));
        # Making the graph undirected:
        edge_data.append([int(target) - 1, int(source) - 1]);
        edge_weight.append(float(weight['weight']));
    # Resulting 32 bit float tensor:
    edge_tensor = torch.tensor(edge_data, dtype=torch.long);
    # Creating weight tensor for edge attribution.
    weight_tensor = torch.tensor(edge_weight, dtype=torch.float);

    return node_tensor, edge_tensor, weight_tensor;

In [ ]:
def integrate_labels_and_graphs(directory=r'dataset/AlphaFold Protein Database e.coli/Uncompressed/graphs/graphML'):
    """
    Combines CSV and graphs into PyTorch Geometric Data objects across
    directory by calling load_amino_acid_labels and load_graph6/sparse6/graphml.

    Parameters
    ----------
    directory : The path to the directory containing the file pairings.
    -Default: r'dataset/AlphaFold Protein Database e.coli/Uncompressed/graphs/graphML'.

    Returns
    -------
    data_list : The geometric data of one-hot encoded amino acid sequences and
    atomic node.
    amino_acids : The list of amino acids

    """

    # Creating a dict for one-hot encoding amino acids to a numeric value.
    amino_acids = [
        'ALA', 'ARG', 'ASN', 'ASP', 'CYS',
        'GLN', 'GLU', 'GLY', 'HIS', 'ILE',
        'LEU', 'LYS', 'MET', 'PHE', 'PRO',
        'SER', 'THR', 'TRP', 'TYR', 'VAL',
        'PYL', 'SEC'
        ];
    AA_onehot = {aa : i for i, aa in enumerate(set(amino_acids))};

    # Initialising array.
    data_list = [];
    # Iterates over the entire directory of file-pairings.
    for file_name in sorted(os.listdir(directory)):
        if file_name.endswith('.csv'):
            # Taking the file name without the extension.
            base_name = file_name.split('AAlabel.csv')[0];
            csv_file = os.path.join(directory, file_name);
            graph_file = os.path.join(directory, f'{base_name}gcn_graph.graphml');
            # Checking the graph file path's validity.
            if os.path.exists(graph_file):
                print(f'Valid path to: {graph_file}');
                AA_labels = load_amino_acid_labels(csv_file, AA_onehot);
                node_list, edge_list, weight_list = load_graph(graph_file);
                data = Data(x=node_list,
                            edge_index=edge_list.t().contiguous(),
                            edge_attr=weight_list.contiguous(),
                            y=AA_labels);
                data_list.append(data);
                if len(data_list) % 10 == 0:
                    print(f'Processed {len(data_list)} entries');
    return data_list;

First model.

In [ ]:
class GNN(MessagePassing):
    def __init__(self):
        super().__init__();
        self.conv1=GATConv(in_channels=-1,
                           out_channels=4096);
        self.conv2=GATConv(in_channels=4096,
                           out_channels=2048);
        self.fc=Linear(in_features=2048,
                        out_features=4);


    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr;
        x = self.conv1(x, edge_index, edge_weight);
        x = relu(x);
        x = self.conv2(x, edge_index, edge_weight);
        x = relu(x);
        return self.fc(x);

In [ ]:
def test_model(loader, model, epoch, epochs, optimizer,
               test, criterion=torch.nn.MSELoss()):

    model.eval();
    total_loss=0;
    for data in loader:
        data = data.to(device);
        if test:
            data_test = deepcopy(data);
            data_test.x * 0;
            data_test = data_test.to(device);
            node_out = model(data_test);
            total_loss += criterion(node_out, data.x);
        else:
            data_val = deepcopy(data);
            if epochs - epoch < 12:
                rank = epoch - (epochs - 12);
                data_val.x[math.ceil(rank/4)] * 0;
                if rank > 4:
                    data_val.x[math.ceil(rank/4) - (math.ceil(rank-4/4) * 2)];
            data_val = data_val.to(device);
            node_out = model(data_val);
            total_loss += criterion(node_out, data.x);
    return total_loss / len(loader);

In [ ]:
def train_model(loader, model, optimizer, criterion=torch.nn.MSELoss()):

    model.train();
    total_loss=0;
    for data in loader:
        # Sending data to the GPU for memory availability purposes.
        data = data.to(device);
        optimizer.zero_grad();
        node_out = model(data);
        loss = criterion(node_out, data.x);
        loss.backward();
        optimizer.step();
        print(f'Loss: {loss.item():.4f}');
        total_loss += loss.item();
    return total_loss / len(loader);

In [ ]:
def process(loader, model, epochs=16,
            chosen_opt=torch.optim.Adam, learn_rate=0.02):

    optimizer=chosen_opt(model.parameters(), lr=learn_rate);
    scheduler=ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.1,
                                patience=4, threshold=100, threshold_mode='abs',
                                cooldown=0, min_lr=0, eps=1e-8);
    model.to(device);
    for epoch in range(epochs):
        avg_loss = 0;
        avg_loss = train_model(loader[0], model, optimizer=optimizer);
        val_mse = test_model(loader[1], model, epoch, epochs,
                             optimizer=optimizer, test=False);
        test_mse = test_model(loader[2], model, epoch, epochs,
                              optimizer=optimizer, test=True);
        print(f'Epoch {epoch+1}, '\
              f'\nAverage MSE training loss: {avg_loss:.4f}'\
                  f'\nAverage MSE validation loss: {val_mse:.4f}'\
                      f'\nAverage MSE test loss: {test_mse:.4f}');
        scheduler.step(val_mse);

Running the program.

In [ ]:
# Setting suitable (GPU) for model training/evaluation, insufficient RAM otherwise.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu');
print(f'Using {device} for training & testing.');

Using cpu for training & testing.


In [ ]:
data_list = integrate_labels_and_graphs(directory=r'/content/drive/MyDrive/Herts_Uni/Final_Project/graphs/graphML');
print(f'Processed {len(data_list)} file pairings into PyTorch Geometric Data objects.');
# Shuffling the Data objects' order in the list.
random.shuffle(data_list);

Valid path to: /content/drive/MyDrive/Herts_Uni/Final_Project/graphs/graphML/AF-A0A385XJ53-F1-model_v4-gcn_graph.graphml
Valid path to: /content/drive/MyDrive/Herts_Uni/Final_Project/graphs/graphML/AF-A0A385XJE6-F1-model_v4-gcn_graph.graphml
Valid path to: /content/drive/MyDrive/Herts_Uni/Final_Project/graphs/graphML/AF-A0A385XJK5-F1-model_v4-gcn_graph.graphml
Valid path to: /content/drive/MyDrive/Herts_Uni/Final_Project/graphs/graphML/AF-A0A385XJL2-F1-model_v4-gcn_graph.graphml
Valid path to: /content/drive/MyDrive/Herts_Uni/Final_Project/graphs/graphML/AF-A0A385XJL4-F1-model_v4-gcn_graph.graphml
Valid path to: /content/drive/MyDrive/Herts_Uni/Final_Project/graphs/graphML/AF-A0A385XJN2-F1-model_v4-gcn_graph.graphml
Valid path to: /content/drive/MyDrive/Herts_Uni/Final_Project/graphs/graphML/AF-A0A385XK32-F1-model_v4-gcn_graph.graphml
Valid path to: /content/drive/MyDrive/Herts_Uni/Final_Project/graphs/graphML/AF-A5A605-F1-model_v4-gcn_graph.graphml
Valid path to: /content/drive/MyDriv

In [ ]:
# Creating parameters for train/validation/test split.
train_ratio, val_ratio, test_ratio = 0.7, 0.2, 0.1;
train_size = int(train_ratio * len(data_list));
val_size = int(val_ratio * len(data_list));
# Creating the split lists.
train_data = data_list[:train_size];
# Sliced from the end of the train set to itself + the validation set.
val_data = data_list[train_size:train_size + val_size];
# Sliced from the end of the validation set to the end of full set.
test_data = data_list[train_size + val_size:];
# Creating the loaders for each split.
train_loader = DataLoader(train_data, batch_size=256);
val_loader = DataLoader(val_data, batch_size=256);
test_loader = DataLoader(test_data, batch_size=256);

In [ ]:
# Training loop for the GNN model.
process(loader=[train_loader, val_loader, test_loader], model=GNN());

Loss: 25762.7773
Loss: 7652650.5000
Loss: 269370.7500
Loss: 124064.9922
Loss: 49598.4219
Loss: 18923.8086
Loss: 12257.8037
Loss: 11562.4277
Loss: 11550.0020
Loss: 12776.2148
Loss: 3460.3579
Loss: 4769.3647
Epoch 1, 
Average MSE training loss: 683062.2851
Average MSE validation loss: 6341.3564
Average MSE test loss: 7419.8965
Loss: 6539.2944
Loss: 1717.9470
Loss: 4100.8682
Loss: 3080.3018
Loss: 1638.4506
Loss: 1328.1199
Loss: 1876.9612
Loss: 673.4135
Loss: 719.6270
Loss: 761.6196
Loss: 632.5350
Loss: 550.2179
Epoch 2, 
Average MSE training loss: 1968.2797
Average MSE validation loss: 458.3136
Average MSE test loss: 425.2726
Loss: 492.8354
Loss: 408.4999
Loss: 416.8718
Loss: 333.3745
Loss: 426.0861
Loss: 310.1480
Loss: 328.9706
Loss: 310.5389
Loss: 286.0248
Loss: 305.5233
Loss: 228.9994
Loss: 255.2627
Epoch 3, 
Average MSE training loss: 341.9280
Average MSE validation loss: 240.0136
Average MSE test loss: 225.4853
Loss: 299.9262
Loss: 196.9433
Loss: 222.1139
Loss: 242.8545
Loss: 222.181